## Transfer learning

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_gen = ImageDataGenerator(rescale=1./255.,
                              rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_gen = ImageDataGenerator(rescale=1./255.)

In [4]:
train_dir='dataset/train_own'
val_dir='dataset/valid_own'

train_generator=train_gen.flow_from_directory(train_dir,target_size=(200,200),class_mode='binary',batch_size=10)
val_generator=val_gen.flow_from_directory(val_dir,target_size=(200,200),class_mode='binary',batch_size=10)

Found 393 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


In [13]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (200, 200, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 10, 10, 768)


In [14]:
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.75)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [15]:
history_transfer = model.fit(
            train_generator,
            validation_data = val_generator,
            steps_per_epoch = 30,
            epochs = 20,
            validation_steps = 5,
            verbose = 2)

Epoch 1/20


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_94/Conv2D}}]]
	 [[loss_3/mul/_3877]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_94/Conv2D}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
from mtcnn.mtcnn import MTCNN
import cv2

img = cv2.imread("dataset/img_test/ojos-cerrados.jpg")
detector = MTCNN()
recog=detector.detect_faces(img)
print(recog)
    
if recog:
    box=recog[0]['box']
    ojo_iz = recog[0]['keypoints']['left_eye']
    ojo_de = recog[0]['keypoints']['right_eye']
    x=box[0]
    y=box[1]
    w=box[2]
    h=box[3]

    ojoiz_x=ojo_iz[0]
    ojoiz_y=ojo_iz[1]

    ojosd_x=ojo_de[0]
    ojosd_y=ojo_de[1]

    ojoiz_new_x = ojoiz_x - 40
    ojoiz_new_y = ojoiz_y - 30
    ojo_w = 70
    ojo_h = 50

    ojode_new_x = ojosd_x - 40
    ojode_new_y = ojosd_y - 30

    img = cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 10)
    img = cv2.rectangle(img, (ojoiz_new_x,ojoiz_new_y), (ojoiz_new_x+ojo_w, ojoiz_new_y+ojo_h), (100,100,100), 2)
    img = cv2.rectangle(img, (ojode_new_x,ojode_new_y), (ojode_new_x+ojo_w, ojode_new_y+ojo_h), (25,100,45), 2)
    #img = cv2.circle(img,(ojoiz_x,ojoiz_y),10,(255,0,255),3)
    #img = cv2.circle(img,(ojosd_x,ojosd_y),20,(255,0,255),3)
    
    cv2.imwrite("dataset/img_test/cerrar-ojos-12.jpg", img)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
crop_ojo_der = img[ojode_new_y:ojode_new_y+ojo_h, ojode_new_x:ojode_new_x+ojo_w]
cv2.imwrite("dataset/img_test/resize.jpg", crop_ojo_der)
output = cv2.resize(crop_ojo_der, (200,200))
x=image.img_to_array(output)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = None
classes = model.predict(images, batch_size=10)
print(classes[0])
    
crop_ojo_iz = img[ojoiz_new_y:ojoiz_new_y+ojo_h, ojoiz_new_x:ojoiz_new_x+ojo_w]
output_1 = cv2.resize(crop_ojo_iz, (200,200)) 
cv2.imwrite("dataset/img_test/resize_1.jpg", output_1)
x_1=image.img_to_array(output_1)
x_1=np.expand_dims(x_1, axis=0)
images_1 = np.vstack([x_1])
classes_1 = None
classes_1 = model.predict(images_1, batch_size=10)
print(classes_1[0])

In [10]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
import tensorflow.keras as keras

vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=(200,200,3))

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)

vgg_model = Model(vgg.input, output)
vgg_model.trainable = False

for layer in vgg_model.layers:
    layer.trainable = False

vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [11]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=(200,200,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 18432)             14714688  
_________________________________________________________________
dense (Dense)                (None, 512)               9437696   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 24,415,553
Trainable params: 9,700,865
Non-trainable params: 14,714,688
____________________________________

In [8]:
from tensorflow.keras.models import load_model

#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('models/model.h5')

In [12]:
history = model.fit_generator(train_generator, steps_per_epoch=40, epochs=60,
                              validation_data=val_generator, validation_steps=9, verbose=1)

Epoch 1/60


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node model_2/block1_conv1/Conv2D}}]]
	 [[loss_2/mul/_1559]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node model_2/block1_conv1/Conv2D}}]]
0 successful operations.
0 derived errors ignored.

In [11]:
model.save('models/model_1.h5')

In [ ]:
from mtcnn.mtcnn import MTCNN
import cv2

img = cv2.imread("dataset/img_test/miedo.jpeg")
detector = MTCNN()
recog=detector.detect_faces(img)
print(recog)
    
if recog:
    box=recog[0]['box']
    ojo_iz = recog[0]['keypoints']['left_eye']
    ojo_de = recog[0]['keypoints']['right_eye']
    x=box[0]
    y=box[1]
    w=box[2]
    h=box[3]

    ojoiz_x=ojo_iz[0]
    ojoiz_y=ojo_iz[1]

    ojosd_x=ojo_de[0]
    ojosd_y=ojo_de[1]

    ojoiz_new_x = ojoiz_x - 40
    ojoiz_new_y = ojoiz_y - 30
    ojo_w = 70
    ojo_h = 50

    ojode_new_x = ojosd_x - 40
    ojode_new_y = ojosd_y - 30
    
    #ojoiz_new_x = ojoiz_x - 230
    #ojoiz_new_y = ojoiz_y - 150
    #ojo_w = 450
    #ojo_h = 300

    #ojode_new_x = ojosd_x - 200
    #ojode_new_y = ojosd_y - 150

    img = cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 10)
    img = cv2.rectangle(img, (ojoiz_new_x,ojoiz_new_y), (ojoiz_new_x+ojo_w, ojoiz_new_y+ojo_h), (100,100,100), 2)
    img = cv2.rectangle(img, (ojode_new_x,ojode_new_y), (ojode_new_x+ojo_w, ojode_new_y+ojo_h), (25,100,45), 2)
    #img = cv2.circle(img,(ojoiz_x,ojoiz_y),10,(255,0,255),3)
    #img = cv2.circle(img,(ojosd_x,ojosd_y),20,(255,0,255),3)
    
    cv2.imwrite("dataset/img_test/cerrar-ojos-12.jpg", img)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
crop_ojo_der = img[ojode_new_y:ojode_new_y+ojo_h, ojode_new_x:ojode_new_x+ojo_w]
cv2.imwrite("dataset/img_test/resize.jpg", crop_ojo_der)
output = cv2.resize(crop_ojo_der, (200,200))
x=image.img_to_array(output)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = None
classes = model.predict(images, batch_size=10)
print(classes[0])
    
crop_ojo_iz = img[ojoiz_new_y:ojoiz_new_y+ojo_h, ojoiz_new_x:ojoiz_new_x+ojo_w]
output_1 = cv2.resize(crop_ojo_iz, (200,200)) 
cv2.imwrite("dataset/img_test/resize_1.jpg", output_1)
x_1=image.img_to_array(output_1)
x_1=np.expand_dims(x_1, axis=0)
images_1 = np.vstack([x_1])
classes_1 = None
classes_1 = model.predict(images_1, batch_size=10)
print(classes_1[0])

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

#Download data
face = "haarcascade.xml"

eye = "eyecascade.xml"

image_l = "dataset/img_test/abierto3.png"


#Classifiers
face_cascade = cv2.CascadeClassifier(face)
eye_cascade = cv2.CascadeClassifier(eye)

#Image we will predict
img = cv2.imread(image_l)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Detect face
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in faces:
    #params = (image, position, width and height, color in RGB scale, and thickness)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    print(x)
    #Detect eyes per face
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        print(ex)
        cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)
        crop_image=roi_color[ey:ey+eh,ex:ex+ew]
        output = cv2.resize(crop_image, (75,75))
        
        x=image.img_to_array(output)
        x=np.expand_dims(x, axis=0)
        images = np.vstack([x])
        classes = None
        classes = model.predict(images, batch_size=10)
        print(classes[0])
        #salida=''
        #if classes[0]==1:
        #    salida=" is a abierto"
        #    break
        #elif classes[0]== 0:
        #    salida=" is a cerrado"
            
#print(salida)

In [ ]:
import numpy as np

from tensorflow.keras.preprocessing import image

path='dataset/img_test/ojos-abiertos.jpg'
img=image.load_img(path, target_size=(100, 100))
  
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=10)
  
print(classes[0])
  
if classes[0]>0:
    print(" is a abierto")
    
else:
    print(" is a cerrado")